In [7]:
%pip install datasets
%pip install git+https://github.com/sylinrl/TruthfulQA
%pip install nltk
%pip install t5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/sylinrl/TruthfulQA to /tmp/pip-req-build-4br3yjqr
  Running command git clone --filter=blob:none --quiet https://github.com/sylinrl/TruthfulQA /tmp/pip-req-build-4br3yjqr
  Resolved https://github.com/sylinrl/TruthfulQA to commit fdd8ad1c0d00a478cf8b0bb41a3ad8378c16293b
  Preparing metadata (setup.py) ... done
  Created wheel for truthfulqa: filename=truthfulqa-0.0.1-py3-none-any.whl size=18615 sha256=d13d963d652c2040163852ea75436cf65514dc59957c1bcda8ab51252cdcb69f
  Stored in directory: /tmp/pip-ephem-wheel-cache-s2po_2b8/wheels/3e/10/ed/872215a52ca279bf40ca8ce50f3f93baa50bd6235a6a497b3f
Successfully built truthfulqa

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
try:
  # for google colab users
    import google.colab # type: ignore
    from google.colab import output
    COLAB = True
    %pip install sae-lens transformer-lens
except:
  # for local setup
    COLAB = False
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

# Imports for displaying vis in Colab / notebook
import webbrowser
import http.server
import socketserver
import threading
PORT = 8000

# general imports
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "2"
import torch
from tqdm import tqdm
import plotly.express as px

torch.set_grad_enabled(False);

In [2]:
# package import
from torch import Tensor
from transformer_lens import utils
from functools import partial
from jaxtyping import Int, Float

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cuda"

print(f"Device: {device}")

Device: cuda


In [3]:
def display_vis_inline(filename: str, height: int = 850):
    '''
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    '''
    if not(COLAB):
        webbrowser.open(filename);

    else:
        global PORT

        def serve(directory):
            os.chdir(directory)

            # Create a handler for serving files
            handler = http.server.SimpleHTTPRequestHandler

            # Create a socket server with the handler
            with socketserver.TCPServer(("", PORT), handler) as httpd:
                print(f"Serving files from {directory} on port {PORT}")
                httpd.serve_forever()

        thread = threading.Thread(target=serve, args=("/content",))
        thread.start()

        output.serve_kernel_port_as_iframe(PORT, path=f"/{filename}", height=height, cache_in_notebook=True)

        PORT += 1

In [4]:
from transformer_lens import HookedTransformer
from sae_lens import SAE
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

# Choose a layer you want to focus on
# For this tutorial, we're going to use layer 2
layer = 16

# get model
model = HookedTransformer.from_pretrained("mistral-7b", device = device)

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release="mistral-7b-res-wg",  # see other options in sae_lens/pretrained_saes.yaml
    sae_id=f"blocks.{layer}.hook_resid_pre",  # won't always be a hook point
    device=device
)

# get hook point
hook_point = sae.cfg.hook_name
print(hook_point)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model mistral-7b into HookedTransformer
blocks.16.hook_resid_pre


In [34]:
from datasets import load_dataset
from truthfulqa.metrics import run_bleu_and_rouge
import pandas as pd

# Load TruthfulQA dataset
truthfulqa_dataset = load_dataset("truthful_qa", "multiple_choice")

def get_truthfulqa_sample():
    # Randomly select a question from the dataset
    sample = truthfulqa_dataset['validation'].shuffle().select(range(1))[0]
    question = sample['question']
    correct_answers = sample['correct_answers']
    incorrect_answers = sample['incorrect_answers']
    return question, correct_answers, incorrect_answers


def evaluate_truthfulness(generated_text, question, correct_answers, incorrect_answers):
    # Create a DataFrame with the generated text and all possible answers
    df = pd.DataFrame({
        'generated': [generated_text] * (len(correct_answers) + len(incorrect_answers)),
        'reference': [format_truthfulqa(question, answer) for answer in correct_answers + incorrect_answers],
        'correct': [1] * len(correct_answers) + [0] * len(incorrect_answers)
    })
    
    # Run BLEU and ROUGE metrics
    results = run_bleu_and_rouge(df['generated'].tolist(), df['reference'].tolist())
    
    # Extract BLEU scores for correct and incorrect answers
    correct_scores = results['bleu'][:len(correct_answers)]
    incorrect_scores = results['bleu'][len(correct_answers):]
    
    # Calculate the score as the difference between max correct and max incorrect scores
    score = max(correct_scores) - max(incorrect_scores)
    
    return score

def compute_reward(original_text, steered_text, correct_answers, incorrect_answers):
    original_score = evaluate_truthfulness(original_text, correct_answers, incorrect_answers)
    steered_score = evaluate_truthfulness(steered_text, correct_answers, incorrect_answers)
    
    # Reward is the improvement in truthfulness score
    reward = steered_score - original_score
    
    return reward

In [35]:
def hooked_generate(prompt_batch, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            stop_at_eos=False,  # avoids a bug on MPS
            input=tokenized,
            max_new_tokens=200,
            do_sample=True,
            **kwargs)
    return result

In [40]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')

def evaluate_truthfulness(generated_text, question, correct_answers, incorrect_answers):
    def calculate_bleu(reference, candidate):
        if isinstance(reference, str):
            reference_tokens = word_tokenize(reference.lower())
        else:
            reference_tokens = [token.lower() for token in reference]
        
        if isinstance(candidate, str):
            candidate_tokens = word_tokenize(candidate.lower())
        else:
            candidate_tokens = [token.lower() for token in candidate]
        
        return sentence_bleu([reference_tokens], candidate_tokens)

    # Calculate BLEU scores for correct answers
    correct_scores = [calculate_bleu(format_truthfulqa(question, answer), generated_text) for answer in correct_answers]
    
    # Calculate BLEU scores for incorrect answers
    incorrect_scores = [calculate_bleu(format_truthfulqa(question, answer), generated_text) for answer in incorrect_answers]
    
    # Calculate the score as the difference between max correct and max incorrect scores
    score = max(correct_scores) - max(incorrect_scores)
    
    return score

def format_truthfulqa(question, answer):
    return f"Q: {question} A: {answer}"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
from transformer_lens import HookedTransformer
from sae_lens import SAE
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
from datasets import load_dataset
from truthfulqa.metrics import run_bleu_and_rouge
import random
import pandas as pd

# Load TruthfulQA dataset
truthfulqa_dataset = load_dataset("truthful_qa", "generation")['validation']

def get_truthfulqa_sample():
    sample = random.choice(truthfulqa_dataset)
    question = sample['question']
    correct_answers = sample['correct_answers']
    incorrect_answers = sample['incorrect_answers']
    return question, correct_answers, incorrect_answers


def compute_reward(original_text, steered_text, question, correct_answers, incorrect_answers):
    # Ensure original_text and steered_text are strings
    if isinstance(original_text, list):
        original_text = ' '.join(original_text)
    if isinstance(steered_text, list):
        steered_text = ' '.join(steered_text)
    
    original_score = evaluate_truthfulness(original_text, question, correct_answers, incorrect_answers)
    steered_score = evaluate_truthfulness(steered_text, question, correct_answers, incorrect_answers)
    reward = steered_score - original_score
    return reward

class ThompsonSamplingClampTuner:
    def __init__(self, sae, alpha=1.0, beta=1.0):
        self.sae = sae
        self.num_features = sae.cfg.d_sae
        self.device = sae.device
        self.dtype = torch.float32
        self.alpha = torch.tensor(alpha, dtype=self.dtype, device=self.device)
        self.beta = torch.tensor(beta, dtype=self.dtype, device=self.device)
        self.theta = torch.zeros(self.num_features, dtype=self.dtype, device=self.device)
        self.precision = torch.ones(self.num_features, dtype=self.dtype, device=self.device)
        
    def choose_action(self, context):
        latent_context = self.sae.encode(context.unsqueeze(0)).squeeze(0)
        sampled_theta = self.sample_parameters()
        return sampled_theta * latent_context
        
    def sample_parameters(self):
        variance = torch.distributions.gamma.Gamma(self.alpha, self.beta).sample((self.num_features,)).to(self.device)
        return torch.normal(self.theta, torch.sqrt(variance))

    
    def update(self, context, action, reward):
        latent_context = self.sae.encode(context.unsqueeze(0)).squeeze(0)
        latent_context = latent_context.to(device=self.device, dtype=self.dtype)
        action = action.to(device=self.device, dtype=self.dtype)
        reward = torch.tensor(reward, dtype=self.dtype, device=self.device)
        self.precision += torch.square(latent_context)
        self.theta += (reward - torch.dot(self.theta, latent_context)) * latent_context / self.precision
        self.alpha += 0.5
        self.beta += 0.5 * (reward - torch.dot(self.theta, latent_context))**2

def thompson_sampling_hook(resid_pre, hook):
    print(f"resid_pre shape: {resid_pre.shape}")
    
    if resid_pre.shape[1] == 1:
        return resid_pre

    # Get the mean activation across the sequence dimension
    context = resid_pre.mean(dim=1).squeeze()
    print(f"context shape: {context.shape}")
    
    # Choose action (steering vector in latent space) based on the context
    latent_steering_vector = ts_tuner.choose_action(context)
    steering_vector = sae.decode(latent_steering_vector.unsqueeze(0)).squeeze(0)
    steering_vector = steering_vector.unsqueeze(0).unsqueeze(0).expand_as(resid_pre)
    result = resid_pre + steering_vector
    
    return result

sampling_kwargs = dict(temperature=1.5, top_p=0.1, freq_penalty=1.0)
def run_generate_with_tuning(num_iterations=10):
    
    for i in range(num_iterations):
        model.reset_hooks()
        
        question, correct_answers, incorrect_answers = get_truthfulqa_sample()
        
        original_res = hooked_generate([question], seed=i, **sampling_kwargs)
        original_text = model.to_string(original_res)
        
        editing_hooks = [(f"blocks.{layer}.hook_resid_post", thompson_sampling_hook)]
        steered_res = hooked_generate([question], editing_hooks, seed=i, **sampling_kwargs)
        steered_text = model.to_string(steered_res)
        
        reward = compute_reward(original_text, steered_text, question, correct_answers, incorrect_answers)
        
        context = model.run_with_cache(question)[1][hook_point].mean(dim=1).squeeze()
        action = thompson_sampling_hook(context.unsqueeze(0).unsqueeze(0), None) - context
        ts_tuner.update(context, action.squeeze(), reward)
        
        print(f"Iteration {i+1}:")
        print(f"Question: {question}")
        print(f"Original response: {original_text}")
        print(f"Steered response: {steered_text}")
        print(f"Reward (truthfulness improvement): {reward:.4f}")
        print()

# Initialize the Thompson Sampling tuner
ts_tuner = ThompsonSamplingClampTuner(sae)

# Run the tuning process
run_generate_with_tuning()


SAE input size: 4096
SAE latent size: 65536
Model hidden size: 4096
Number of layers: 32
Selected layer for steering: 16


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

resid_pre shape: torch.Size([1, 13, 4096])
context shape: torch.Size([4096])
Input context shape: torch.Size([4096])
Latent context shape: torch.Size([65536])
Sampled theta shape: torch.Size([65536])
latent_steering_vector shape: torch.Size([65536])
steering_vector shape: torch.Size([4096])
Reshaped steering_vector shape: torch.Size([1, 13, 4096])
result shape: torch.Size([1, 13, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

resid_pre shape: torch.Size([1, 11, 4096])
context shape: torch.Size([4096])
Input context shape: torch.Size([4096])
Latent context shape: torch.Size([65536])
Sampled theta shape: torch.Size([65536])
latent_steering_vector shape: torch.Size([65536])
steering_vector shape: torch.Size([4096])
Reshaped steering_vector shape: torch.Size([1, 11, 4096])
result shape: torch.Size([1, 11, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

resid_pre shape: torch.Size([1, 9, 4096])
context shape: torch.Size([4096])
Input context shape: torch.Size([4096])
Latent context shape: torch.Size([65536])
Sampled theta shape: torch.Size([65536])
latent_steering_vector shape: torch.Size([65536])
steering_vector shape: torch.Size([4096])
Reshaped steering_vector shape: torch.Size([1, 9, 4096])
result shape: torch.Size([1, 9, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Size([1, 1, 4096])
resid_pre shape: torch.Si

In [38]:
%pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [44]:
example_prompt = "Who actually said 'let them eat cake'"

run_generate_with_tuning(example_prompt)

Model hidden size: 4096
Number of layers: 32
Selected layer for steering: 16


TypeError: 'str' object cannot be interpreted as an integer